
## 1. Download the pre-trained InceptionV3 model

In [ ]:
import warnings
warnings.filterwarnings('ignore')

from keras.preprocessing import image
from keras.applications.inception_v3 import InceptionV3

import numpy as np
import os

conv_base = InceptionV3(weights = 'imagenet', include_top = False, input_shape=(150,150,3))

## 2. Create a model (using a pre-trained model)

In [ ]:
from keras import models
from keras import layers
conv_base.trainable = False
model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(3, activation='softmax'))

model.summary()

In [ ]:
from keras import optimizers
model.compile(loss = 'categorical_crossentropy', optimizer = optimizers.RMSprop(lr=1e-4), metrics=['acc'])

## 3. Load training and validation datasets

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_dir = 'images/ponies/train'
validation_dir = 'images/ponies/validate'

train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range = 40,
        width_shift_range = 0.2,
        height_shift_range = 0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode = 'nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(150, 150),
        batch_size=16,
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=16,
        class_mode='categorical')

label_map = (train_generator.class_indices)

## 4. Train and save the model

In [ ]:
history = model.fit_generator(
        train_generator,
        steps_per_epoch=20,
        epochs=10,
        validation_data=validation_generator,
        validation_steps=10) 

In [ ]:
model.save('my_little_pony_transfer.h5')

## 5. Use created model for prediction

In [ ]:
from keras.preprocessing import image
import matplotlib.image as mpimg
import numpy as np
from keras.models import load_model

def predict_image(image_url):
  img = image.load_img(img_path, target_size=(150, 150))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis = 0)
  x /= 255

  model = load_model('my_little_pony_transfer.h5')

  # retrieve the most likely result, e.g. highest probability

  y_prob = model.predict(x) 
  y_classes = y_prob.argmax(axis=-1)

  class_name = [k for k, v in label_map.items() if v == y_classes[0]]
  # print the classification
  print('Prediction: class: %s, score: (%.5f)' % (class_name[0], y_prob[0][0]))

In [ ]:
import os

img_folder = "images/ponies/test/"
print('------------------- TRANSFER LEARNING ----------------------------------')
for filename in os.listdir(img_folder):
      img_path = os.path.join(img_folder,filename)
      print ('processing image:', img_path)
      predict_image(img_path)
      print('-----------------------------------------------------')

The end!